In [1]:
#from google.cloud import secretmanager
import pandas as pd
import numpy as np
from currensee.utils.db_utils import create_pg_engine
from sqlalchemy import text

import pandas as pd
import random
import os
import re



2025-07-13 15:50:20,307 - currensee.core.secrets - INFO - PROJECT_ID not found, attempting to load from .env file
2025-07-13 15:50:20,311 - currensee.core.secrets - INFO - Loaded environment from ./.env
2025-07-13 15:50:20,312 - currensee.core.secrets - INFO - SecretManager initialized with project_id: adsp-34002-on02-sopho-scribe


In [2]:
# Define DB_NAME
DB_NAME = "crm_outlook"

In [3]:
engine = create_pg_engine(db_name=DB_NAME)

In [4]:
pd.read_sql("SELECT * FROM preferences limit 10", con=engine)

,as_of_date,employee_first_name,employee_last_name,email,title,finance_detail,news_detail,macro_news_detail,past_meeting_detail
0,2023-01-23 10:00:00,Jane,Moneypenny,jane.moneypenny@bankwell.com,Relationship Manager,full,full,full,full
1,2023-01-23 10:00:00,Monica,Melton,monica.melton@bankwell.com,Product Specialist,none,full,full,short
2,2023-01-23 10:00:00,Michael,Long,michael.long@bankwell.com,Finance Assistant,full,full,full,short
3,2023-01-23 10:00:00,Breanna,Guerra,breanna.guerra@bankwell.com,Senior Relationship Manager,short,none,full,none
4,2023-01-23 10:00:00,Heather,Singh,heather.singh@bankwell.com,Financial Advisor,full,none,full,none
5,2023-01-23 10:00:00,Alejandro,Walters,alejandro.walters@bankwell.com,Senior Relationship Manager,none,full,full,short
6,2023-01-23 10:00:00,Ryan,Holloway,ryan.holloway@bankwell.com,Relationship Manager,none,full,none,full
7,2023-01-23 10:00:00,Anthony,Reynolds,anthony.reynolds@bankwell.com,Financial Advisor,short,full,full,none
8,2023-01-23 10:00:00,Alexandra,Terry,alexandra.terry@bankwell.com,Product Specialist,none,full,none,full
9,2023-01-23 10:00:00,Jordan,Robles,jordan.robles@bankwell.com,Financial Advisor,full,none,full,none


In [10]:
pd.read_sql("""SELECT as_of_date
, employee_first_name
, employee_last_name
, finance_detail
, news_detail
, macro_news_detail
, past_meeting_detail 
FROM preferences 
where employee_first_name = 'Ryan' and employee_last_name = 'Holloway' 
order by as_of_date
""", con=engine)

,as_of_date,employee_first_name,employee_last_name,finance_detail,news_detail,macro_news_detail,past_meeting_detail
0,2023-01-23 10:00:00,Ryan,Holloway,none,full,none,full
1,2023-04-02 10:00:00,Ryan,Holloway,short,full,full,none
2,2023-07-26 14:00:00,Ryan,Holloway,short,none,none,short
3,2023-11-04 10:00:00,Ryan,Holloway,short,none,full,full
4,2024-01-30 10:00:00,Ryan,Holloway,none,full,none,none
5,2024-03-10 09:00:00,Ryan,Holloway,none,none,none,short


In [9]:
pd.read_sql("""SELECT as_of_date
, employee_first_name
, employee_last_name
, finance_detail  
, news_detail
, macro_news_detail
, past_meeting_detail 
FROM preferences 
where employee_first_name = 'Jane' and employee_last_name = 'Moneypenny' 
order by as_of_date
""", con=engine)

,as_of_date,employee_first_name,employee_last_name,finance_detail,news_detail,macro_news_detail,past_meeting_detail
0,2023-01-23 10:00:00,Jane,Moneypenny,full,full,full,full
1,2023-04-02 10:00:00,Jane,Moneypenny,short,full,full,short
2,2023-07-26 14:00:00,Jane,Moneypenny,none,none,full,short
3,2023-11-04 10:00:00,Jane,Moneypenny,short,full,none,none
4,2024-01-30 10:00:00,Jane,Moneypenny,short,full,none,full
5,2024-03-10 09:00:00,Jane,Moneypenny,full,none,full,full


In [6]:
df = pd.read_sql("SELECT * from employees", con=engine)
df.head()

,index,employee_id,first_name,last_name,title,email,phone,hire_date,department,market
0,0,31e735f6-cfbe-4107-8aba-4ef45a90e9ae,Jane,Moneypenny,Relationship Manager,jane.moneypenny@bankwell.com,971.841.4711x92417,2021-01-14,Enterprise Investment,San Fransisco
1,1,b61a7a99-791a-4b9f-923f-271810defac4,Monica,Melton,Product Specialist,monica.melton@bankwell.com,884.541.7837,2021-12-03,Small Business Investment,Chicago
2,2,2f8c004d-e0a6-4bba-956e-2b60af632674,Michael,Long,Finance Assistant,michael.long@bankwell.com,478-831-1994x378,2021-12-06,Sales,Washington DC
3,3,59dcd404-9d5c-47f4-9ba1-2de5156343f8,Breanna,Guerra,Senior Relationship Manager,breanna.guerra@bankwell.com,736-597-4792x589,2024-07-11,Enterprise Investment,Boston
4,4,7a650dd8-34a3-4148-a7f8-631bf5db37d4,Heather,Singh,Financial Advisor,heather.singh@bankwell.com,7139112404,2020-11-15,Customer Support,Washington DC


In [11]:
df = pd.read_sql("""SELECT p.as_of_date
        , p.employee_first_name
        , p.employee_last_name
        , p.finance_detail
        , p.news_detail
        , p.macro_news_detail
        , p.past_meeting_detail 
        , p.email	
        FROM preferences p
        where p.email = 'jane.moneypenny@bankwell.com'
        """, con=engine)
df.head(10)

,as_of_date,employee_first_name,employee_last_name,finance_detail,news_detail,macro_news_detail,past_meeting_detail,email
0,2023-01-23 10:00:00,Jane,Moneypenny,full,full,full,full,jane.moneypenny@bankwell.com
1,2023-04-02 10:00:00,Jane,Moneypenny,short,full,full,short,jane.moneypenny@bankwell.com
2,2023-07-26 14:00:00,Jane,Moneypenny,none,none,full,short,jane.moneypenny@bankwell.com
3,2023-11-04 10:00:00,Jane,Moneypenny,short,full,none,none,jane.moneypenny@bankwell.com
4,2024-01-30 10:00:00,Jane,Moneypenny,short,full,none,full,jane.moneypenny@bankwell.com
5,2024-03-10 09:00:00,Jane,Moneypenny,full,none,full,full,jane.moneypenny@bankwell.com


In [15]:
user_email = df["email"][0]
user_email

'jane.moneypenny1@bankwell.com'

In [22]:

mx_dt_df['as_of_date'][0]

'2024-01-30 10:00:00'

In [23]:
user_email = df["email"][0]


mx_dt_df = pd.read_sql(f"""
    SELECT max(p.as_of_date) as as_of_date
    FROM preferences p
    where p.email = '{user_email}'
    and as_of_date <= '{meeting_timestamp}'
""", con=engine)

max_dt = mx_dt_df['as_of_date'][0]

query_str = f"""
    SELECT p.as_of_date
    , p.employee_first_name
    , p.employee_last_name
    , p.finance_detail
    , p.news_detail
    , p.macro_news_detail
    , p.past_meeting_detail 
    , e.email	
    FROM preferences p
    inner join employees e
    on p.employee_first_name = e.first_name 
    and p.employee_last_name = e.last_name 
    where e.email = '{user_email}' and p.as_of_date = '{max_dt}'
"""

pref_df = pd.read_sql(query_str, con=engine)


In [29]:
pref_df.head()

,as_of_date,employee_first_name,employee_last_name,finance_detail,news_detail,macro_news_detail,past_meeting_detail,email
0,2024-01-30 10:00:00,Jane,Moneypenny,True,True,False,True,jane.moneypenny1@bankwell.com


In [28]:
fin_detail = pref_df["finance_detail"][0]
news_detail = pref_df["news_detail"][0]
macro_news_detail = pref_df["macro_news_detail"][0]
past_meeting_detail = pref_df["past_meeting_detail"][0]

print(fin_detail, news_detail, macro_news_detail, past_meeting_detail)

True True False True


In [53]:
user_email = df["email"][0]
meeting_timestamp = "2024-02-04 14:30:00"
mx_dt_df = pd.read_sql(f"""
    SELECT max(p.as_of_date) as as_of_date
    FROM preferences p
    where p.email = '{user_email}'
    and as_of_date <= '{meeting_timestamp}'
    """, con=engine)
    
max_dt = mx_dt_df['as_of_date'][0]

query_str = f"""
SELECT p.as_of_date
, p.employee_first_name
, p.employee_last_name
, p.finance_detail
, p.news_detail
, p.macro_news_detail
, p.past_meeting_detail 
, p.email	
FROM preferences p
where p.email = '{user_email}' and p.as_of_date = '{max_dt}'
"""
pref_df = pd.read_sql(query_str, con=engine)

In [54]:
print(max_dt)

2024-01-30 10:00:00


In [55]:
pref_df.head()

,as_of_date,employee_first_name,employee_last_name,finance_detail,news_detail,macro_news_detail,past_meeting_detail,email
0,2024-01-30 10:00:00,Jane,Moneypenny,short,full,none,full,jane.moneypenny1@bankwell.com


In [10]:
df = pd.read_sql("""SELECT * from meeting_data where 
 host_email = 'jane.moneypenny@bankwell.com'
and invitee_emails	= 'adam.clay@compass.com'
--and meeting_timestamp > '2023-01-23 10:00:00' 

order by meeting_timestamp desc """, con=engine)
df.head(30)

,index,meeting_timestamp,host,host_email,invitees,invitee_emails,meeting_subject
0,93.0,2024-03-26 11:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Annual Credit Facility Review Meeting
1,51.0,2021-10-01 11:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Credit Facility Documentation Final ...
2,34.0,2020-09-15 14:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Credit Facility Term Sheet Follow-up...
3,18.0,2019-09-17 10:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Credit Facility Term Sheet Review
4,NaN,2018-12-19 15:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Compass,adam.clay@compass.com,Compass - Benchmark Change
5,NaN,2018-12-05 15:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Compass,adam.clay@compass.com,Compass - New Funds Offerings
6,NaN,2018-11-21 15:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Compass,adam.clay@compass.com,Compass - Macro Update
7,NaN,2018-11-07 15:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Compass,adam.clay@compass.com,Compass - ESG Investing
8,NaN,2018-10-24 15:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Compass,adam.clay@compass.com,Compass - Tax Optimization
9,NaN,2018-10-10 15:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Compass,adam.clay@compass.com,Compass - Annual Goal Alignment


In [21]:


query_str = f"""
    SELECT  meeting_timestamp
    FROM  meeting_data
    WHERE  invitee_emails = 'adam.clay@compass.com'
    and host_email = 'jane.moneypenny@bankwell.com'
    and meeting_timestamp < '2024-03-26 11:00:00'
    ORDER BY meeting_timestamp DESC
    
    LIMIT 1;"""



last_meeting = pd.read_sql(query_str, con=engine)

last_meeting["meeting_timestamp"][0]

'2021-10-01 11:00:00'

In [6]:
query_str = f"""
    SELECT  *
    FROM  meeting_data
    WHERE  invitee_emails = 'adam.clay@compass.com'
    and host_email = 'jane.moneypenny@bankwell.com'
    ORDER BY meeting_timestamp DESC;"""



last_meeting = pd.read_sql(query_str, con=engine)

last_meeting.head(20)

,index,meeting_timestamp,host,host_email,invitees,invitee_emails,meeting_subject
0,NaN,2025-06-16 11:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - AML Requirements & Policy Developments
1,NaN,2025-06-11 14:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Tracking Error & Volatility Shift Study
2,NaN,2025-06-11 12:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Geopolitical Risks & Global Supply E...
3,NaN,2025-06-01 13:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Low-Volatility Fund Enhancements
4,NaN,2025-05-25 14:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - ESG Ratings Dashboard Review
5,NaN,2025-05-21 10:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Charitable Deduction & Gift Strategy
6,NaN,2025-05-17 16:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Strategic Risk Metrics & Tolerance R...
7,NaN,2025-05-17 16:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Capital Allocation & Milestone Mapping
8,93.0,2024-03-26 11:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Annual Credit Facility Review Meeting
9,51.0,2021-10-01 11:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Compass - Credit Facility Documentation Final ...


In [ ]:
query_str = f"""
    SELECT  *
    FROM  meeting_data
    WHERE  invitee_emails = 'adam.clay@compass.com'
    and host_email = 'jane.moneypenny@bankwell.com'
    ORDER BY meeting_timestamp DESC;"""



last_meeting = pd.read_sql(query_str, con=engine)

last_meeting.head(20)

In [22]:
qry = """ SELECT *
        FROM clients_contact
        --WHERE company = '{client_company}'
        """
df = pd.read_sql(qry, con=engine)

In [23]:
df.head()

,company,industry,account_id,contact_first_name,contact_last_name,contact_title,phone,email,website,location,annual_revenue,total_account_bal
0,Mariott,Hospitality,d02ccad1-26c4-4768-95a4-833dbc2b2460,Mary,Vasquez,Senior Director,520-618-5303x572,mary.vasquez@mariott.com,mariott.com,New York City,43000000,30000000
1,AbbVie,Healthcare,d02ccad1-26c4-4768-95a4-833dbc2b2461,Cynthia,Hobbs,Director,(446)673-8121x90878,cynthia.hobbs@abbvie.com,AbbVie.com,Chicago,35000000,39000000
2,AeroVironment,Manufacturing,d02ccad1-26c4-4768-95a4-833dbc2b2462,Jennifer,Phelps,Senior Director,797-584-0061x89137,jennifer.phelps@aerovironment.com,AeroVironment.com,Washington DC,25000000,4000000
3,Amedisys,Healthcare,c42e62e8-e943-47e0-bfd1-651e053c8bc6,Kyle,Waters,Senior Director,590.239.3215x8014,kyle.waters@amedisys.com,Amedisys.org,Washington DC,36891234,45213789
4,Celestica,Technology,c42e62e8-e943-47e0-bfd1-651e053c8bc5,Denise,Moore,Senior Director,754-579-1511x763,denise.moore@celestica.com,celestica.com,Washington DC,36891234,45213789


In [10]:
qry = """SELECT email_timestamp
        , to_names	
        , to_emails
        , from_name
        , from_email
        , email_subject
        , email_body
        FROM email_data
        WHERE email_timestamp > '2024-03-26 11:00:00'
        and ( to_emails = 'adam.clay@compass.com' OR from_email = 'adam.clay@compass.com')
        and (to_emails = 'jane.moneypenny@bankwell.com' or from_email = 'jane.moneypenny@bankwell.com')
        limit 15
        """
df = pd.read_sql(qry, con=engine)
df.head()

,email_timestamp,to_names,to_emails,from_name,from_email,email_subject,email_body
0,2025-05-15 19:00:00,Adam Clay,adam.clay@compass.com,Jane Moneypenny,jane.moneypenny@bankwell.com,Compass - Strategic Risk Metrics & Tolerance R...,"Hello Adam Clay,\n\nStress testing scenarios w..."
1,2025-05-16 03:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Re: Compass - Strategic Risk Metrics & Toleran...,"Dear Jane,\n\nThank you for your email. Hello ..."
2,2025-05-16 03:00:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Re: Compass - Strategic Risk Metrics & Toleran...,"Dear Jane,\n\nThank you for your email. Hello ..."
3,2025-05-17 03:15:00,Adam Clay,adam.clay@compass.com,Jane Moneypenny,jane.moneypenny@bankwell.com,Compass - Capital Allocation & Milestone Mapping,"Hi Adam Clay,\n\nCapital allocation and milest..."
4,2025-05-17 13:30:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Adam Clay,adam.clay@compass.com,Re: Compass - Capital Allocation & Milestone M...,"Dear Jane,\n\nThank you for your email. Hi Jan..."


In [19]:
meeting_timestamp = '2024-03-26 11:00:00'
user_email = 'jane.moneypenny@bankwell.com'
query_str = f"""
    SELECT email_timestamp
        , to_names	
        , to_emails
        , from_name
        , from_email
        , email_subject
        , email_body
        FROM email_data
        WHERE email_timestamp < '{meeting_timestamp}'
        and to_emails = '{user_email}'
        order by email_timestamp desc
        limit 20
"""
result = pd.read_sql(query_str, con=engine)
recent_emails = list(result["email_body"])   

In [21]:
result.head()

,email_timestamp,to_names,to_emails,from_name,from_email,email_subject,email_body
0,2024-01-24 16:10:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Jessica Palmer,jessica.palmer@hasbro.com,Hasbro - Post Divestiture Strategy Adjustments,"Hi jane,\n\nYes, this needs further thought. I..."
1,2024-01-21 09:17:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Jessica Palmer,jessica.palmer@hasbro.com,Hasbro - Post Divestiture Strategy Adjustments,"Thanks Jane,\n\nThank you for the update. Seem..."
2,2023-12-13 10:15:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Michelle Jenkins,michelle.jenkins@intuitivesurgical.com,Re: Intuitive Surgical - Discussing FX Service...,"Hi Jane,\n\nYes, let's plan for that in Q1. Ho..."
3,2023-12-12 11:30:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Jessica Palmer,jessica.palmer@hasbro.com,Re: Hasbro - Strategy Discussion for H1 2024 F...,"Thanks Jane,\n\nI will follow up if I have any..."
4,2023-12-05 11:30:00,Jane Moneypenny,jane.moneypenny@bankwell.com,Timothy Ochoa,timothy.ochoa@hyatthotels.com,Re: Hyatt Hotels - 401(k) Advisory Services RFI,"Hi Jane,\n\nThanks for checking in. The RFI pr..."


In [18]:
result_dict = result.to_dict()
result_dict

{'email_timestamp': {0: '2018-07-15 09:30:00',
  1: '2018-07-16 11:05:00',
  2: '2018-07-16 11:15:00',
  3: '2018-08-01 14:00:00',
  4: '2018-08-02 10:15:00',
  5: '2018-08-02 16:30:00',
  6: '2018-08-10 11:00:00',
  7: '2018-08-13 09:45:00',
  8: '2018-08-13 10:00:00',
  9: '2018-09-05 15:00:00',
  10: '2018-09-06 10:30:00',
  11: '2018-09-06 14:20:00',
  12: '2018-09-18 09:00:00',
  13: '2018-09-18 13:15:00',
  14: '2018-09-18 13:30:00'},
 'to_names': {0: 'Cynthia Hobbs',
  1: 'Jane Moneypenny',
  2: 'Cynthia Hobbs',
  3: 'Jennifer Phelps',
  4: 'Jane Moneypenny',
  5: 'Jennifer Phelps',
  6: 'Kyle Waters',
  7: 'Jane Moneypenny',
  8: 'Kyle Waters',
  9: 'Denise Moore',
  10: 'Jane Moneypenny',
  11: 'Denise Moore',
  12: 'Adam Clay',
  13: 'Jane Moneypenny',
  14: 'Adam Clay'},
 'to_emails': {0: 'cynthia.hobbs@abbvie.com',
  1: 'jane.moneypenny@bankwell.com',
  2: 'cynthia.hobbs@abbvie.com',
  3: 'jennifer.phelps@aerovironment.com',
  4: 'jane.moneypenny@bankwell.com',
  5: 'jennif

In [11]:
recent_emails

["Hi Cynthia,\\n\\nMy name is Jane Moneypenny, and I'm a Financial Advisor with Bankwell Financial, specializing in corporate financial solutions for the Healthcare sector. I came across your profile and wanted to reach out to see if AbbVie might benefit from our expertise in cash management, investment strategies, and risk mitigation.\\n\\nWould you be open to a brief introductory call sometime next week?\\n\\nBest regards,\\n\\n--\\nJane Moneypenny\\nFinancial Advisor\\nBankwell Financial\\nPhone: (555) 123-4567\\nEmail: jane.moneypenny1@outlook.com\\n",
 "Hi Jane,\\n\\nThank you for reaching out. AbbVie is always interested in optimizing its financial operations. I'd be happy to schedule a brief call. How does Tuesday at 2 PM CT sound?\\n\\nBest,\\n\\n--\\nCynthia Hobbs\\nDirector\\nAbbVie\\nEmail: cynthia.hobbs@abbvie.com\\nPhone: (446)673-8121x90878\\n",
 "Hi Cynthia,\\n\\nTuesday at 2 PM CT works perfectly. I'll send a calendar invitation shortly. Looking forward to our conversat